# Dressmaker - Easy

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@192.168.31.31:15432/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 60)


 ········


In [2]:
jmcust = pd.read_sql_table('jmcust', engine)
dressmaker = pd.read_sql_table('dressmaker', engine)
dress_order = pd.read_sql_table('dress_order', engine)
construction = pd.read_sql_table('construction', engine)
quantities = pd.read_sql_table('quantities', engine)
order_line = pd.read_sql_table('order_line', engine)
garment = pd.read_sql_table('garment', engine)
material = pd.read_sql_table('material', engine)

## 1.
List the post code, order number, order date and garment descriptions for all items associated with Ms Brown.

In [3]:
t1 = (jmcust.loc[jmcust['c_name'].str.strip()=='Ms Brown']
      .merge(dress_order, left_on='c_no', right_on='cust_no')
      .merge(order_line, left_on='order_no', right_on='order_ref')
      .merge(garment, left_on='ol_style', right_on='style_no')
      .rename(columns={'c_post_code': 'post_code'})
      [['post_code', 'order_no', 'order_date', 'description']])
t2 = (dressmaker.loc[dressmaker['d_name'].str.strip()=='Ms Brown']
      .merge(construction, left_on='d_no', right_on='maker')
      .merge(dress_order, left_on='order_ref', right_on='order_no')
      .merge(order_line, left_on='order_no', right_on='order_ref')
      .merge(garment, left_on='ol_style', right_on='style_no')
      .rename(columns={'d_post_code': 'post_code'})
      [['post_code', 'order_no', 'order_date', 'description']])
pd.concat([t1, t2])

,post_code,order_no,order_date,description
0,B2 5AB,7,2002-02-21,Short Skirt
1,B2 5AB,7,2002-02-21,Sundress
2,B2 5AB,7,2002-02-21,Suntop
3,B2 5AB,9,2002-02-27,Shorts


## 2.
List the customer name, postal information, order date and order number of all orders that have been completed.

In [4]:
(jmcust.merge(dress_order.loc[dress_order['completed']=='Y'],
              left_on='c_no', right_on='cust_no')
 .merge(order_line, left_on='order_no', right_on='order_ref')
 [['c_name', 'c_post_code', 'order_date', 'order_no']])

,c_name,c_post_code,order_date,order_no
0,Ms White,E24 8PQ,2002-02-03,5
1,Ms White,E24 8PQ,2002-02-03,5
2,Ms White,E24 8PQ,2002-02-03,5
3,Mr Brass,FG24 9NM,2002-02-02,4
4,Mr Brass,FG24 9NM,2002-02-02,4
5,Ms Muir,H2 7CV,2002-01-20,3
6,Dr Green,SJ4 4WE,2002-01-11,2
7,Dr Green,SJ4 4WE,2002-01-11,2
8,Dr Green,SJ4 4WE,2002-01-11,2
9,Mrs Peacock,DD6 9NM,2002-01-10,1


## 3.
Which garments have been made or are being made from 'red abstract' or 'blue abstract' coloured materials.

In [5]:
(garment.merge(order_line, left_on='style_no', right_on='ol_style')
 .merge(material.loc[material['colour'].str.strip()
        .str.lower().isin(['red abstract', 'blue abstract'])],
        left_on='ol_material', right_on='material_no')
 [['style_no', 'description', 'colour']])

,style_no,description,colour
0,1,Trousers,Red Abstract
1,2,Long Skirt,Red Abstract
2,5,Sundress,Red Abstract
3,2,Long Skirt,Blue Abstract
4,4,Short Skirt,Blue Abstract


## 4.
How many garments has each dressmaker constructed? You should give the number of garments and the name and postal information of each dressmaker.

In [6]:
(dressmaker.merge(construction, left_on='d_no', right_on='maker')
 .merge(dress_order, left_on='order_ref', right_on='order_no')
 .merge(order_line, left_on='order_no', right_on='order_ref')
 .merge(garment, left_on='ol_style', right_on='style_no')
 .groupby(['d_no', 'd_name', 'd_post_code'])
 .agg(cnt=pd.NamedAgg(column='line_no', aggfunc='count'))
 .reset_index())

,d_no,d_name,d_post_code,cnt
0,1,Mrs Hem,A12 6BC,14
1,2,Miss Stitch,DF4 7HJ,20
2,3,Mr Needles,E12 6LG,12
3,4,Ms Sew,EF7 9KL,12
4,5,Mr Seam,H45 7YH,12
5,6,Mr Taylor,SH6 9RT,7
6,7,Miss Pins,B4 9BL,11


## 5.
Give the names of those dressmakers who have finished items made from silk for completed orders.

In [7]:
(dressmaker.merge(construction, left_on='d_no', right_on='maker')
 .merge(dress_order.loc[dress_order['completed']=='Y'], 
        left_on='order_ref', right_on='order_no')
 .merge(order_line, left_on='order_no', right_on='order_ref')
 .merge(garment, left_on='ol_style', right_on='style_no')
 .merge(material.loc[material['fabric'].str.strip().str.lower()=='silk'], 
        left_on='ol_material', right_on='material_no')
 [['d_name']].drop_duplicates())

,d_name
0,Mrs Hem
1,Miss Stitch
5,Miss Pins
7,Mr Needles
8,Ms Sew
